In [1]:
import pandas as pd
import nibabel as nib
import numpy as np
import os
from dice import dice

from utils import organizeFolders, organizeFiles, openImage

In [5]:
# Main Folder Dir
fp = 'C:\\Users\\colli\\Desktop\\lesionInterfaces\\Ground Truth Analysis'

# Pandas dataframe to store everything
"""
Shorthand meanings
    - 'vol' = original image volume
    - 'GT' = ground truth segmentation
    - 'MS' = Manual Segmentation
    - 'SC' = Smart Click Segmentation
"""
df = pd.DataFrame(columns=[
    # Patient ID
    'patient_id',

    # Voxel Count
    'vol_voxCnt',
    'GT_voxCnt',
    'MS_voxCnt',
    'SC_voxCnt',

    # Dice Score
    'GT-MS_dice',
    'GT-SC_dice',
    'MS_SC_dice',
])

# Maping file names to respective categories
file_map = {
    'vol' : 'dicom_vol',
    'GT' : 'GT_liver_cyst_vol',
    'MS' : 'DR_liver_cyst_vol',
    'SC' : 'DR_liver_cyst_vol_smart_click'}

inv_file_map = {v: k for k, v in file_map.items()}

In [11]:
df = pd.DataFrame()

# Get folder paths
folder_dir = organizeFolders(fp)

# Get file paths and info
for i in folder_dir:
    # Get File Paths
    file_dir = organizeFiles(i)
    patient_id = i.split('\\')[-1]
    print(patient_id)

    # Organize all numpy arrays into dictionary following the file_map
    np_files = {}
    np_files_vol = {}

    for j in file_dir:
        np_arr, volume = openImage(j)
        np_files[ inv_file_map[j.split('\\')[-1].split('.nii')[0]] ] = np_arr
        np_files_vol[ inv_file_map[j.split('\\')[-1].split('.nii')[0]] ] = volume


    print(np_files_vol)

    # Start geting information
    ## voxCnt and volume

    temp = {}
    keyword = '_voxCnt'
    keyword2 = '_volume(in cm^3)'
    for k in np_files.keys():
        temp['patient_id'] = patient_id
        if k == 'vol':
            temp[k+keyword] = round(np_files[k].shape[0] * np_files[k].shape[1] * np_files[k].shape[2])
            temp[k+keyword2] = round(np_files_vol[k] * temp[k+keyword], 6)
        else:
            temp[k+keyword] = round(np_files[k].sum())
            temp[k+keyword2] = round(np_files_vol[k] * temp[k+keyword], 6)

    # Dice Score
    temp['GT-MS_dice'] = dice(np_files['GT'], np_files['MS']) # Ground Truth - Manual Segmentation
    temp['GT-SC_dice'] = dice(np_files['GT'], np_files['SC']) # Ground Truth - Smart Click Segmentation
    temp['MS_SC_dice'] = dice(np_files['MS'], np_files['SC']) # Manual Segmentation - Smart Click Segmentation


    df = df.append(pd.Series(temp, name=patient_id), ignore_index=True)

        

DEBUG: list directories - ['WC-IRB1308014251_003', 'WC-IRB1308014251_008', 'WC-IRB1308014251_011', 'WC-IRB1308014251_020', 'WC-IRB1308014251_025', 'WC-IRB1308014251_030']
WC-IRB1308014251_003
{'vol': 67.734375, 'MS': 67.734375, 'SC': 67.734375, 'GT': 67.734375}
WC-IRB1308014251_008
{'vol': 93.75, 'MS': 93.75, 'SC': 93.75, 'GT': 93.75}
WC-IRB1308014251_011
{'vol': 185.66349029541016, 'MS': 185.66349029541016, 'SC': 185.66349029541016, 'GT': 185.66349029541016}
WC-IRB1308014251_020
{'vol': 113.4375, 'MS': 113.4375, 'SC': 113.4375, 'GT': 113.4375}
WC-IRB1308014251_025
{'vol': 109.86714363098145, 'MS': 109.86714363098145, 'SC': 109.86714363098145, 'GT': 109.86714363098145}
WC-IRB1308014251_030
{'vol': 84.609375, 'MS': 84.609375, 'SC': 84.609375, 'GT': 84.609375}


In [12]:
df

,patient_id,vol_voxCnt,vol_volume(in cm^3),MS_voxCnt,MS_volume(in cm^3),SC_voxCnt,SC_volume(in cm^3),GT_voxCnt,GT_volume(in cm^3),GT-MS_dice,GT-SC_dice,MS_SC_dice
0,WC-IRB1308014251_003,6323200.0,4.282980e+08,4810.0,3.258023e+05,4719.0,3.196385e+05,5862.0,3.970589e+05,0.788043,0.888763,0.755378
1,WC-IRB1308014251_008,5529600.0,5.184000e+08,51396.0,4.818375e+06,49033.0,4.596844e+06,52839.0,4.953656e+06,0.985811,0.918820,0.921566
2,WC-IRB1308014251_011,4456448.0,8.273997e+08,82829.0,1.537832e+07,71072.0,1.319548e+07,74369.0,1.380761e+07,0.888154,0.976465,0.869390
3,WC-IRB1308014251_020,5529600.0,6.272640e+08,10136.0,1.149802e+06,9929.0,1.126321e+06,10851.0,1.230910e+06,0.901510,0.955438,0.891104
4,WC-IRB1308014251_025,2031616.0,2.232078e+08,11486.0,1.261934e+06,11649.0,1.279842e+06,13167.0,1.446621e+06,0.826999,0.937379,0.809077
5,WC-IRB1308014251_030,5529600.0,4.678560e+08,4670.0,3.951258e+05,5158.0,4.364152e+05,7565.0,6.400699e+05,0.587985,0.808772,0.545381


In [42]:
df.to_csv('dice_score.csv')